In [4]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import os

In [5]:
!unzip -uq "/content/drive/MyDrive/datasets/3rd dts.zip" -d "data/"

In [6]:
data = pd.read_csv("data/esc50.csv")
data.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [7]:
labels = list(data['category'].unique())

In [11]:
path = '/content/data/audio/audio/'

In [ ]:
def features_extractor(file):
    audio, sr = librosa.load(file)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    return mfccs_scaled_features
data['extracted_feature'] = data['filename'].apply(lambda x: features_extractor(path + x))

In [14]:
data.head()

,filename,fold,target,category,esc10,src_file,take,extracted_feature
0,1-100032-A-0.wav,1,0,dog,True,100032,A,"[-600.9402, 4.69429, -8.5059395, -4.2766213, -..."
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A,"[-193.9782, 2.7461467, -59.386883, 4.41346, -4..."
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A,"[16.259264, 56.921246, -8.960626, 15.122986, -..."
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B,"[17.771097, 54.86499, -7.1793256, 13.654805, -..."
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A,"[-423.07523, 124.92982, 38.760685, 35.04086, -..."


In [21]:
data['category'].unique().size

50

In [27]:
new_data = data[['extracted_feature','category']]

In [28]:
new_data.head()

,extracted_feature,category
0,"[-600.9402, 4.69429, -8.5059395, -4.2766213, -...",dog
1,"[-193.9782, 2.7461467, -59.386883, 4.41346, -4...",chirping_birds
2,"[16.259264, 56.921246, -8.960626, 15.122986, -...",vacuum_cleaner
3,"[17.771097, 54.86499, -7.1793256, 13.654805, -...",vacuum_cleaner
4,"[-423.07523, 124.92982, 38.760685, 35.04086, -...",thunderstorm


In [36]:
X=np.array(new_data['extracted_feature'].tolist())
y=np.array(new_data['category'].tolist())

In [32]:
X.shape

(2000, 40)

In [39]:
y = np.array(pd.get_dummies(y))

In [40]:
y.shape

(2000, 50)

In [41]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state = 42)

In [43]:
import tensorflow as tf

In [44]:
num_labels = y.shape[1]

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(40,)),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation='softmax')
])

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 200)               8200      
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 200)               40200     
                                                                 
 dropout_2 (Dropout)         (None, 200)               0         
                                                                 
 dense_4 (Dense)             (None, 50)                10050     
                                                                 
Total params: 58450 (228.32 KB)
Trainable params: 58450 (228.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stopping_cb])

Epoch 1/100
50/50 [==============================] - 2s 22ms/step - loss: 1.5707 - accuracy: 0.5487 - val_loss: 2.5276 - val_accuracy: 0.4125
Epoch 2/100
50/50 [==============================] - 0s 5ms/step - loss: 1.4580 - accuracy: 0.5638 - val_loss: 2.5887 - val_accuracy: 0.3800
Epoch 3/100
50/50 [==============================] - 0s 6ms/step - loss: 1.4003 - accuracy: 0.5869 - val_loss: 2.6029 - val_accuracy: 0.4275
Epoch 4/100
50/50 [==============================] - 0s 6ms/step - loss: 1.3748 - accuracy: 0.5969 - val_loss: 2.5828 - val_accuracy: 0.4125
Epoch 5/100
50/50 [==============================] - 0s 6ms/step - loss: 1.3596 - accuracy: 0.6112 - val_loss: 2.6106 - val_accuracy: 0.3900
Epoch 6/100
50/50 [==============================] - 0s 4ms/step - loss: 1.3245 - accuracy: 0.6062 - val_loss: 2.6672 - val_accuracy: 0.4075
Epoch 7/100
50/50 [==============================] - 0s 4ms/step - loss: 1.3102 - accuracy: 0.6206 - val_loss: 2.6026 - val_accuracy: 0.4025
Epoch 8/100


test_accuracy = model.evaluate(X_test,y_test,verbose =0)
print(test_accuracy[1])

In [55]:
test_accuracy = model.evaluate(X_test,y_test,verbose =0)
print(test_accuracy[1])

0.4124999940395355
